In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Algorithm.py'))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Environment.py'))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Replay_Buffer.py'))))
import Algorithm as ALGS
import Environment as ENVS
import Replay_Buffer


In [2]:
# import collections

# def initialize_buffer(config):
#     return SimpleExperienceBuffer(config['max_size'], config['batch_size'])

# class SimpleExperienceBuffer:
#     def __init__(self, capacity, batch_size):
#         self.buffer = collections.deque(maxlen=capacity)
#         self.batch_size = batch_size

#     def __len__(self):
#         return len(self.buffer)

#     def append(self, experience):
#         self.buffer.append(experience)

#     def sample(self):
#         indices = np.random.choice(len(self.buffer), self.batch_size)
#         states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
#         return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
#                np.array(dones, dtype=np.uint8), np.array(next_states)

# Implementation Example

## First initialize stuff

In [3]:
config ={'Learner': {'type': 'DDPG', 'episodes': 200}, 'Algorithm': {'algorithm': 'DDPG', 'replay_buffer': True, 'learning_rate': 0.003, 'optimizer': 'Adam', 'loss_function': 'MSELoss', 'regularizer': 0, 'recurrence': 0, 'gamma': 0.99, 'beta': 0, 'epsilon_start': 1, 'epsilon_end': 0.02, 'epsilon_decay': 5e-05, 'c': 200}, 'Environment': {'env_type': 'Gym', 'environment': 'MountainCar-v0', 'action_space': 'discrete', 'observation_space': 'discrete', 'env_render': True, 'num_agents': 1}, 'Replay_Buffer': {'max_size': 100000, 'batch_size': 64, 'num_agents': 1}, 'Agent': {'num_agents': 1}, 'Network': {'network_actor': {'layers': '400,300', 'activation_function': 'ReLU', 'output_function': 'Tanh', 'last_layer': True}, 'network_critic_head': {'layers': '400', 'activation_function': 'ReLU', 'output_function': '', 'last_layer': False}, 'network_critic_tail': {'layers': '300', 'activation_function': 'ReLU', 'output_function': '', 'last_layer': True}}}

In [4]:
config

{'Learner': {'type': 'DDPG', 'episodes': 200},
 'Algorithm': {'algorithm': 'DDPG',
  'replay_buffer': True,
  'learning_rate': 0.003,
  'optimizer': 'Adam',
  'loss_function': 'MSELoss',
  'regularizer': 0,
  'recurrence': 0,
  'gamma': 0.99,
  'beta': 0,
  'epsilon_start': 1,
  'epsilon_end': 0.02,
  'epsilon_decay': 5e-05,
  'c': 200},
 'Environment': {'env_type': 'Gym',
  'environment': 'MountainCar-v0',
  'action_space': 'discrete',
  'observation_space': 'discrete',
  'env_render': True,
  'num_agents': 1},
 'Replay_Buffer': {'max_size': 100000, 'batch_size': 64, 'num_agents': 1},
 'Agent': {'num_agents': 1},
 'Network': {'network_actor': {'layers': '400,300',
   'activation_function': 'ReLU',
   'output_function': 'Tanh',
   'last_layer': True},
  'network_critic_head': {'layers': '400',
   'activation_function': 'ReLU',
   'output_function': '',
   'last_layer': False},
  'network_critic_tail': {'layers': '300',
   'activation_function': 'ReLU',
   'output_function': '',
   'las

In [5]:
env = ENVS.initialize_env(config['Environment'])
env

In [6]:
alg = ALGS.initialize_algorithm(env.get_observation_space(), env.get_action_space(), [config['Algorithm'], config['Agent'], config['Network']])
alg

In [7]:
print (alg)

In [8]:
agent = alg.create_agent()
agent

In [9]:
obs = env.get_observations()
obs

array([-0.42967872,  0.        ])

In [10]:
t = torch.tensor(obs)
t

tensor([-0.4297,  0.0000], dtype=torch.float64)

In [11]:
agent.critic.net_head

Sequential(
  (0): Linear(in_features=2, out_features=400, bias=True)
  (1): ReLU()
)

In [12]:
agent.critic.net_tail

Sequential(
  (0): Linear(in_features=403, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=1, bias=True)
)

In [13]:
agent.actor.net

Sequential(
  (0): Linear(in_features=2, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=3, bias=True)
  (5): Tanh()
)

In [14]:
agent.target_actor.net

Sequential(
  (0): Linear(in_features=2, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=3, bias=True)
  (5): Tanh()
)

In [15]:
alg.get_action(agent, t.float(), 1)

this is the action -> [-0.00512713  0.17242543  0.04476905]


/home/phu/Documents/GitHub/shiva2/Control-Tasks/Shiva/Algorithm.py:360: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action = agent.actor(torch.tensor(observation).float()).data.numpy() + self.ou_noise.noise()


array([-0.00512713,  0.17242543,  0.04476905])

In [16]:
buffer = Replay_Buffer.initialize_buffer(config['Replay_Buffer'], None, None, None)
buffer

## Now simulate some training

In [17]:
num_of_episodes = 50

for i in range(num_of_episodes):
    obs = env.reset()
    obs = env.get_observation()
    done = False
    while not done:
        obs
        action = alg.get_action(agent, obs, i)
        next_obs, reward, done = env.step(action)
        
        experience = [obs, action, reward, done, next_obs]
        buffer.append(experience)
        
        experience = buffer.sample()
        alg.update(agent, experience, i)

        obs = next_obs

this is the action -> [0.0030724  0.21660139 0.03917872]
this is the action -> [-0.09710109 -0.45078285 -0.24589882]
this is the action -> [-0.29862073 -0.76970834 -0.48383274]


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

this is the action -> [-0.54693464 -0.89333786 -0.683276  ]
this is the action -> [-0.73942219 -0.93049439 -0.86443601]
this is the action -> [-0.8917783  -0.94996883 -0.97127094]
this is the action -> [-0.9964418  -0.92690111 -1.01383223]
this is the action -> [-1.05768932 -0.92210329 -1.04736854]
this is the action -> [-1.04922351 -0.93447893 -1.06622322]
this is the action -> [-1.04982538 -0.9415722  -1.06073155]
this is the action -> [-1.04580158 -0.95301674 -1.05572091]


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


this is the action -> [-1.06976353 -0.95596795 -1.04992674]
this is the action -> [-1.05623099 -0.97296953 -1.04439734]
this is the action -> [-1.06062023 -0.99197    -1.02980657]
this is the action -> [-1.05057813 -1.00014865 -1.06543244]
this is the action -> [-0.99755258 -1.04253349 -1.02478204]
this is the action -> [-0.97091434 -1.03218626 -1.02576059]
this is the action -> [-0.98368524 -1.03064494 -1.01083684]
this is the action -> [-1.00150124 -1.0083042  -1.00063892]
this is the action -> [-1.02685315 -1.04516727 -0.9946297 ]
this is the action -> [-1.07450092 -1.07639449 -0.99650136]


...

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

this is the action -> [-1.04345138 -1.0567829  -0.92892478]
this is the action -> [-1.00561307 -1.06085375 -0.96238003]
this is the action -> [-0.95289365 -1.03909607 -0.99444703]
this is the action -> [-0.97570871 -1.02451725 -1.04648296]
this is the action -> [-0.99232926 -1.04266213 -1.05990131]
this is the action -> [-0.98981212 -1.03800079 -1.03997554]
this is the action -> [-0.98047622 -1.02189658 -1.01675255]
this is the action -> [-1.00960888 -1.01736276 -1.01588654]
this is the action -> [-1.0097934  -1.00597155 -0.99274164]
this is the action -> [-1.01117437 -0.98944014 -0.98380356]
this is the action -> [-1.00119278 -1.02647381 -1.01073195]
this is the action -> [-0.99579978 -0.98420974 -1.00214731]
this is the action -> [-1.00764821 -1.00222103 -1.01561645]
this is the action -> [-1.03105261 -0.97530946 -1.0019917 ]
this is the action -> [-1.0181365  -0.96692554 -1.02875824]


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

this is the action -> [-1.03547977 -0.95497611 -1.0636702 ]
this is the action -> [-1.04193804 -0.98529771 -1.0669695 ]
this is the action -> [-1.02844729 -0.970917   -1.03213933]
this is the action -> [-1.01475214 -0.9653324  -1.03176798]
this is the action -> [-1.00677155 -0.99336312 -1.06736755]
this is the action -> [-1.01323429 -0.96407705 -1.0317874 ]
this is the action -> [-1.02409534 -0.93628636 -1.03712635]
this is the action -> [-1.01428977 -0.96785206 -1.05639352]
this is the action -> [-1.00259783 -0.98368987 -1.03084999]
this is the action -> [-1.00170511 -0.95515959 -1.02355952]
this is the action -> [-1.02944677 -1.00210601 -1.04199767]


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


In [18]:
obs, actions, rewards, dones, next_obs = buffer.sample()
obs.shape, actions.shape, rewards.shape, dones.shape, next_obs.shape

((64, 2), (64, 3), (64,), (64,), (64, 2))

In [19]:
def action2one_hot_v(action: int):
    z = torch.zeros(4)
    z[action] = 1
    return z

In [20]:
def action2one_hot(action: int):
    z = np.zeros(4)
    z[action] = 1
    return z

In [21]:
i = 1
act = action2one_hot(i)
type(act)

numpy.ndarray

In [22]:
act2 = action2one_hot_v(i)
act2

tensor([0., 1., 0., 0.])

In [23]:
states = [ np.random.rand(4) for i in range(4) ]
actions = [ action2one_hot(np.random.randint(4)) for i in range(4) ]

In [24]:
states

[array([0.37744341, 0.92315005, 0.02220947, 0.69904616]),
 array([0.16501355, 0.61774626, 0.22714334, 0.32380404]),
 array([0.92384349, 0.79031416, 0.65618073, 0.65524334]),
 array([0.33950091, 0.44989515, 0.2670891 , 0.29960915])]

In [25]:
actions

[array([0., 0., 1., 0.]),
 array([0., 0., 0., 1.]),
 array([1., 0., 0., 0.]),
 array([1., 0., 0., 0.])]

In [26]:
input_v = torch.tensor([ np.concatenate([s_i, a_i]) for s_i, a_i in zip(states, actions) ])
input_v

tensor([[0.3774, 0.9232, 0.0222, 0.6990, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.1650, 0.6177, 0.2271, 0.3238, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.9238, 0.7903, 0.6562, 0.6552, 1.0000, 0.0000, 0.0000, 0.0000],
        [0.3395, 0.4499, 0.2671, 0.2996, 1.0000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [27]:
net = torch.nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

In [28]:
state_action_val = net(input_v.float())

In [29]:
dones = [True, False, False, True]
done_mask = torch.ByteTensor(dones)#.to(device)
done_mask

tensor([1, 0, 0, 1], dtype=torch.uint8)

In [30]:
state_action_val[done_mask] = 0.00
state_action_val

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


tensor([[0.0000],
        [0.0732],
        [0.0974],
        [0.0000]], grad_fn=<IndexPutBackward>)